# 2.1 파이토치 개요

### 2.1.1 PyTorch 특징 및 장점

Python은 컴파일을 하지 않고 인터프리터로 한줄 한줄 실행시키기 때문에 실행속도가 느리다.
또한 기본적 연산들이 전부 CPU에서 처리되고, 기본적으로 한번에 1가지 연산밖에 하지 못한다.

> PyTorch는 "GPU에서 텐서 조작 및 동적 신경망 구축이 가능한 프레임워크"이다.

* GPU: GPU를 이용하여 많은 계산을 병렬적으로 빠르게 처리 가능하다. 특히 gradient 계산을 자동으로, 빠르게 해준다.
* 텐서: 텐서 연산을 지원한다. (~np.ndarray)
* 동적 신경망: 훈련 반복시마다 네트워크 변경 가능

장점은 다음과 같습니다:
* 단순함(효율적인 계산)
    * 파이썬 환경과 쉽게 통합할 수 있다
    * 디버깅이 직관적이고 간결하다

* 성능(낮은 CPU 활용)
    * 모델 훈련을 위한 CPU 사용률이 TF에 비해 낮음
    * 학습 및 추론 속도가 빠르고 다루기 쉽다

* 직관적인 인터페이스
    * TF 처럼 잦은 API 변경이 없어 배우기 쉽다

### 2.1.2 PyTorch의 아키텍처
###

<img src="KakaoTalk_20240402_171042585.jpg" alt="Alternative text" width = "700"/>



* PyTorch API : 사용자가 사용하기 쉬운 인터페이스 제공, 실제 계산은 X
    * torch: GPU 연산 지원 텐서 패키지
    * torch.autograd: 자동 미분 패키지
    * torch.nn: 신경망 구축 및 훈련 패키지. Convolution, RNN, Normalize 등등 다양한 내장
    * torch.multiprocessing: 파이토치 프로세스 전판의 메모리 공유가 가능함
    * torch.utils: DataLoader(훈련 데이터 관리), 기타 함수들, 유틸리티 제공
#
* PyTorch 엔진:
    * Autograd C++: weight, bias 업데이트에 필요한 gradient 자동 계산
    * Aten C++: 텐서 라이브러리 제공
    * JIT C++: 계산을 최적화하기 위한 JIT(Just In-Time) 컴파일러

### 2.2.5 모델 훈련
###
<img src="KakaoTalk_20240402_185553622.jpg" alt="Alternative text" width = "900"/>

#
모델을 정의했으면, loss func.을 정의하고, 이 loss func.을 최소화시키는 파라미터 $(w,b)$를 찾습니다.
이 과정을 '훈련(train)' 또는 '학습(learning)'이라고 부릅니다.

이전의 장에서 우리는 optimizer에 모델의 파라미터들을 넣어줬습니다.

In [ ]:
for epoch in range(100):
    yhat = model(x_train)
    # train 데이터를 모델의 forward()에 입력합니다.
    # 연산 과정은 torch.autograd를 통해 자동으로 저장됩니다.
    loss = criterion(yhat, y_train)
    # 지정한 loss func.에 참값(yhat)과 모델의 추정값(y_train)을 집어넣어 loss를 구합니다.
    optimizer.zero.grad()
    # 이전에 optimizer에 누적된 gradient 값들을 초기화시킵니다.
    loss.backward()
    # 역전파 학습 (기울기를 계산)
    optimizer.step()
    # 기울기 업데이트. 하나의 step을 나간 것.

### 2.2.6 모델 평가
###
학습된(중인) 모델을 평가하기 위해서는 테스트 데이터셋을 이용합니다.
본 교재는 torchmetrics 라는 모듈을 이용해 모델을 평가합니다.

In [10]:
import torch
import torchmetrics

preds = torch.randn(10,5).softmax(dim=-1)
target = torch.randint(5,(10,))

acc = torchmetrics.functional.accuracy(preds, target, task="multiclass", num_classes=5)
acc

tensor(0.1000)

### 모듈을 이용하여 모델을 평가하기

In [17]:
metric = torchmetrics.Accuracy(task="multiclass", num_classes=5) # 모델 정확도 초기화

n_batches = 10
for i in range(n_batches):
    preds = torch.randn(10,5).softmax(dim=-1)
    target = torch.randint(5,(10,))

    acc = metric(preds, target)
    print(f"Accuracy on batch {i}: {acc}")

acc = metric.compute()
print(f"Accuracy on all data: {acc}")

Accuracy on batch 0: 0.20000000298023224
Accuracy on batch 1: 0.30000001192092896
Accuracy on batch 2: 0.20000000298023224
Accuracy on batch 3: 0.10000000149011612
Accuracy on batch 4: 0.10000000149011612
Accuracy on batch 5: 0.20000000298023224
Accuracy on batch 6: 0.0
Accuracy on batch 7: 0.20000000298023224
Accuracy on batch 8: 0.20000000298023224
Accuracy on batch 9: 0.4000000059604645
Accuracy on all data: 0.1899999976158142


이 외에도 사이킷런의 metrics 모듈의 confusion_matrix, accuracy_score, classification_report 클래스 등을 이용할 수 있다.

### 2.2.7 훈련 과정 모니터링
###
오늘날 머신러닝/딥러닝은 파라미터의 개수가 상상을 초월할 정도로 많고, 훈련 중 각 파라미터의 변화를 모니터링하는 것은 쉽지 않다.
텐서보드(Tensorboard)는 학습에 사용되는 각종 파라미터 값들의 변화를 시각화시켜 보여준다.

사용 방법은 다음과 같다.
1. 텐서보드를 설정(set up)한다.
2. 텐서보드에 기록(write)한다.
3. 텐서보드를 사용하여 모델 구조를 살펴본다.

패키지 이름은: 'tensorboard'이다.

발표자가 직접 하면서, TensorFlow(pip 상에서 'tensorflow')가 설치되어야 정상적으로 동작한다.

In [3]:
import torch
import torch.nn as nn
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

In [4]:

class C3_layer(nn.Module):
    def __init__(self):
        super(C3_layer, self).__init__()
        self.ch_in_3 = [[0, 1, 2],
                        [1, 2, 3],
                        [2, 3, 4],
                        [3, 4, 5],
                        [0, 4, 5],
                        [0, 1, 5]] # filter with 3 subset of input channels
        self.ch_in_4 = [[0, 1, 2, 3],
                        [1, 2, 3, 4],
                        [2, 3, 4, 5],
                        [0, 3, 4, 5],
                        [0, 1, 4, 5],
                        [0, 1, 2, 5],
                        [0, 1, 3, 4],
                        [1, 2, 4, 5],
                        [0, 2, 3, 5]] # filter with 4 subset of input channels
        # put implementation here
        self.ch_in_6 = [0, 1, 2, 3, 4, 5]

        self.Conv3 = nn.ModuleList()
        self.Conv4 = nn.ModuleList()

        for _ in range(6):
          self.Conv3.append(nn.Conv2d(3,1,kernel_size=5))

        for _ in range(9):
          self.Conv4.append(nn.Conv2d(4,1,kernel_size=5))

        self.Conv6 = nn.Conv2d(6,1,kernel_size=5)

    def forward(self, x):
        # put implementation here
        output3, output4 = list(), list()
        for i, flt in enumerate(self.Conv3):
          output3.append(flt(x[:,self.ch_in_3[i],:,:]))
        for i, flt in enumerate(self.Conv4):
          output4.append(flt(x[:,self.ch_in_4[i],:,:]))
        output6 = self.Conv6(x[:,self.ch_in_6,:,:])

        #print(output3[0].shape)
        #print(output4[0].shape)
        #print(output6.shape)

        cat_out = torch.cat((*output3,*output4,output6),1)
        return cat_out

class LeNet(nn.Module) :
    def __init__(self) :
        super(LeNet, self).__init__()
        #padding=2 makes 28x28 image into 32x32
        self.C1_layer = nn.Sequential(
                nn.Conv2d(1, 6, kernel_size=5, padding=2),
                nn.Tanh()
                )
        self.P2_layer = nn.Sequential(
                nn.AvgPool2d(kernel_size=2, stride=2),
                nn.Tanh()
                )
        self.C3_layer = nn.Sequential(
                C3_layer(),
                nn.Tanh()
                )
        self.P4_layer = nn.Sequential(
                nn.AvgPool2d(kernel_size=2, stride=2),
                nn.Tanh()
                )
        self.C5_layer = nn.Sequential(
                nn.Linear(5*5*16, 120),
                nn.Tanh()
                )
        self.F6_layer = nn.Sequential(
                nn.Linear(120, 84),
                nn.Tanh()
                )
        self.F7_layer = nn.Linear(84, 10)
        self.tanh = nn.Tanh()

    def forward(self, x) :
        output = self.C1_layer(x)
        output = self.P2_layer(output)
        output = self.C3_layer(output)
        output = self.P4_layer(output)
        output = output.view(-1,5*5*16)
        output = self.C5_layer(output)
        output = self.F6_layer(output)
        output = self.F7_layer(output)
        return output




In [18]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter() # 저장할 위치


model = LeNet().to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)

import time
start = time.time()
for epoch in range(1) :
    print("{}th epoch starting.".format(epoch))
    for images, labels in train_loader :
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        train_loss = loss_function(model(images), labels)

        writer.add_scalar("Loss", train_loss, epoch) # 스칼라 값 기록

        train_loss.backward()

        optimizer.step()
end = time.time()
print("Time ellapsed in training is: {}".format(end - start))


0th epoch starting.
Time ellapsed in training is: 48.00064969062805


In [ ]:
writer.close()

<img src="tensorboard1.PNG" alt="Alternative text" width = "900"/>

진짜 이거 안돼서 2시간을 삽질했습니다...